In [14]:
from pycox.evaluation import EvalSurv
import torch
import torchtuples as tt
import os
import pickle
from lifelines.utils import concordance_index

from utils import dataset_name
from eval_utils import MODEL_CLASS, get_results, DL_MODELS, ML_MODELS, MODELS
import numpy as np
import pandas as pd
np.random.seed(42)
_ = torch.manual_seed(42)


print(f"Experiments over {dataset_name}")

Experiments over FLCHAIN


In [15]:
train = pd.read_csv(f"../../datasets/train/{dataset_name}.csv")
test = pd.read_csv(f"../../datasets/test/{dataset_name}.csv")

X, y = train.drop(["event", "duration"], axis=1), train[["event", "duration"]]
X_test, y_test = test.drop(["event", "duration"], axis=1), test[["event", "duration"]]

# Finding the best parameters for each model and run the models over the test data

In [16]:
results = pd.DataFrame()
     
for model in MODELS:
    stats = pickle.load( open( f"statistics/{model}/best_model.pkl", "rb" ))
    df = pd.DataFrame(stats, index=[model])
    results = pd.concat([results, df], axis=0)
    
c_index_df = results[["c_index", "c_index_std", "c_index_params"]].sort_values("c_index", ascending=False)
concordance_td_df = results[["concordance_td", "concordance_td_std", "concordance_td_params"]].sort_values("concordance_td", ascending=False)
ibs_df = results[["ibs", "ibs_std", "ibs_params"]].sort_values("ibs", ascending=True)

## `c index`

In [17]:
c_index_df

,c_index,c_index_std,c_index_params
deep_surv,0.797311,0.013732,deep_surv_dropout_0.1_num_nodes_[100]_activati...
gbst,0.792264,0.014550,gbst_learning_rate_0.1_n_estimators_100_subsam...
reg_coxph,0.790805,0.011331,reg_coxph_l1_ratio_0.2_tol_1e-05_max_iter_1000...
rsf,0.788299,0.017836,rsf_n_estimators_200_max_depth_10_min_samples_...
cox_time,0.784701,0.008043,"cox_time_dropout_0.1_num_nodes_[32, 128, 128]_..."
pc_hazard,0.675961,0.046388,"pc_hazard_dropout_0.3_num_nodes_[256, 256]_act..."
deep_hit,0.253414,0.009952,"deep_hit_dropout_0.1_num_nodes_[16, 32, 64, 64..."


In [18]:
results = []
column = "c_index_params"


for model_name, params in zip(c_index_df.index, c_index_df[column]):
    if model_name in DL_MODELS:
        c_index, concordance_td, ibs = get_results(params, model_name, X, y, X_test, y_test)
        results.append((model_name, c_index))
    else:
        c_index, concordance_td, ibs = get_results(params, model_name, X, y, X_test, y_test)
        results.append((model_name, c_index))

for m,val in sorted(results, key=lambda x: x[1], reverse=True):
    print(m, np.round(val,4))

reg_coxph 0.7968
deep_surv 0.7951
rsf 0.795
gbst 0.7922
cox_time 0.7839
deep_hit 0.7341
pc_hazard 0.2487


___

## `concordance td`

In [19]:
concordance_td_df

,concordance_td,concordance_td_std,concordance_td_params
deep_surv,0.797311,0.013732,deep_surv_dropout_0.1_num_nodes_[100]_activati...
deep_hit,0.794666,0.010514,"deep_hit_dropout_0.3_num_nodes_[28, 28, 100, 2..."
gbst,0.792071,0.014577,gbst_learning_rate_0.1_n_estimators_100_subsam...
reg_coxph,0.790805,0.011331,reg_coxph_l1_ratio_0.2_tol_1e-05_max_iter_1000...
rsf,0.784042,0.017322,rsf_n_estimators_200_max_depth_5_min_samples_s...
cox_time,0.783394,0.012919,"cox_time_dropout_0.1_num_nodes_[32, 128, 128]_..."
pc_hazard,0.639115,0.043577,pc_hazard_dropout_0.3_num_nodes_[100]_activati...


In [20]:
results = []
column = "concordance_td_params"


for model_name, params in zip(concordance_td_df.index, concordance_td_df[column]):
    if model_name in DL_MODELS:
        c_index, concordance_td, ibs = get_results(params, model_name, X, y, X_test, y_test)
        results.append((model_name, concordance_td))
    else:
        c_index, concordance_td, ibs = get_results(params, model_name, X, y, X_test, y_test)
        results.append((model_name, concordance_td))

for m,val in sorted(results, key=lambda x: x[1], reverse=True):
    print(m, np.round(val,4))

reg_coxph 0.7968
deep_surv 0.7951
gbst 0.7919
rsf 0.7898
deep_hit 0.7851
cox_time 0.7633
pc_hazard 0.3158


## `integrated brier score`

In [21]:
ibs_df

,ibs,ibs_std,ibs_params
pc_hazard,0.026777,0.003477,"pc_hazard_dropout_0.3_num_nodes_[16, 16, 16, 6..."
deep_surv,0.096429,0.005028,deep_surv_dropout_0.1_num_nodes_[100]_activati...
reg_coxph,0.098747,0.004502,reg_coxph_l1_ratio_0.1_tol_0.1_max_iter_1000_v...
gbst,0.098943,0.004602,gbst_learning_rate_0.1_n_estimators_100_subsam...
rsf,0.098962,0.004451,rsf_n_estimators_200_max_depth_10_min_samples_...
cox_time,0.106136,0.006701,"cox_time_dropout_0.1_num_nodes_[32, 128, 128]_..."
deep_hit,0.375623,0.023328,"deep_hit_dropout_0.3_num_nodes_[3, 5]_activati..."


In [22]:
results = []
column = "ibs_params"


for model_name, params in zip(ibs_df.index, ibs_df[column]):
    if model_name in DL_MODELS:
        c_index, concordance_td, ibs = get_results(params, model_name, X, y, X_test, y_test)
        results.append((model_name, ibs))
    else:
        c_index, concordance_td, ibs = get_results(params, model_name, X, y, X_test, y_test)
        results.append((model_name, ibs))

for m,val in sorted(results, key=lambda x: x[1], reverse=False):
    print(m, np.round(val,4))

reg_coxph 0.1022
gbst 0.1029
rsf 0.1032
deep_surv 0.1035
cox_time 0.1223
deep_hit 0.1791
pc_hazard 0.7933
